In [2]:
import urllib.request, json, csv, geopy.distance
import pandas as pd
import numpy as np
from datetime import datetime
from time import sleep, strptime, mktime
apikey = ''

In [3]:
GPS_resource_id = 'f2e5503e-927d-4ad3-9500-4ab9e55deb59'
resorce_type = ('1', '2')
GPS_columns = ['Lines', 'VehicleNumber', 'Brigade', 'Time', 'Lon', 'Lat',\
               'res_type', 'qTime', 'd_time', 'd_dist', 'coord', 'uniqID', 'vehID']
last_GPS_df = pd.DataFrame([], columns=GPS_columns)

def delta_time(vehID, new_time):
    last_time = last_GPS_df[last_GPS_df.vehID == vehID]['Time']
    if (len(last_time)>0):
        last_time = strptime(last_time.values[0], "%Y-%m-%d %H:%M:%S")
        new_time = strptime(new_time, "%Y-%m-%d %H:%M:%S")
        return int(mktime(new_time) - mktime(last_time))
    return -1

def delta_dist(vehID, new_coord):
    last_coord = last_GPS_df[last_GPS_df.vehID == vehID]['coord']
    if (len(last_coord)>0):
        return round(geopy.distance.distance(last_coord, new_coord).kilometers, 5)
    return -1

In [4]:
while True:
    sleep(5)
    new_GPS_df = pd.DataFrame([], columns=GPS_columns)
    for one in resorce_type:
        api = f'https://api.um.warszawa.pl/api/action/busestrams_get/?apikey={apikey}&type={one}&resource_id={GPS_resource_id}'
        with urllib.request.urlopen(api) as url:
            dataGPS = json.loads(url.read().decode())['result']
        if(type(dataGPS)==str): #Błędna metoda lub parametry wywołania
            continue
        df = pd.DataFrame(dataGPS, columns=GPS_columns)       
        df['coord'] = df[ ['Lon', 'Lat'] ].apply(lambda x: \
                (pd.to_numeric(x['Lat'], errors='coerce'), pd.to_numeric(x['Lon'], errors='coerce')), axis=1)        
        df['res_type'] = one
        df['qTime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        df['uniqID'] = df[ ['Lines', 'VehicleNumber', 'Brigade', 'Time'] ]\
            .apply(lambda x: (x['Lines']+'_'+x['VehicleNumber']+'_'+x['Brigade']+'_'+x['Time']), axis=1)
        df['vehID'] = df[ ['Lines', 'VehicleNumber', 'Brigade'] ]\
            .apply(lambda x: (x['Lines']+'_'+x['VehicleNumber']+'_'+x['Brigade']), axis=1)
        new_GPS_df = pd.concat([new_GPS_df, df])
    uniq_items = new_GPS_df[~new_GPS_df.uniqID.isin(last_GPS_df.uniqID)]
    if(len(uniq_items)==0): 
        continue
    uniq_items['d_time'] = uniq_items[ ['vehID', 'Time'] ].apply(lambda x: delta_time(x['vehID'], x['Time']), axis=1)
    uniq_items['d_dist'] = uniq_items[ ['vehID', 'coord'] ].apply(lambda x: delta_dist(x['vehID'], x['coord']), axis=1)
    #uniq_items['mean_veloc'] = 
    last_GPS_df = pd.concat([last_GPS_df, new_GPS_df])
    last_GPS_df.drop_duplicates(subset=['vehID'], inplace=True, keep='last')
    print(uniq_items.shape, last_GPS_df.shape, new_GPS_df.shape)
    with open('dataGPS.csv', 'a') as f:
        uniq_items.iloc[:,0:10].to_csv(f, header=False, index=False, sep=',')

(944, 13) (944, 13) (944, 13)


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(362, 13) (946, 13) (943, 13)
(358, 13) (948, 13) (945, 13)
(382, 13) (948, 13) (942, 13)
(46, 13) (948, 13) (941, 13)
(334, 13) (948, 13) (942, 13)
(439, 13) (949, 13) (942, 13)
(362, 13) (949, 13) (940, 13)
(45, 13) (949, 13) (939, 13)
(369, 13) (949, 13) (938, 13)
(311, 13) (949, 13) (937, 13)


KeyboardInterrupt: 